In [ ]:
# Basic setup and sanity checking
%run ./00_setup.ipynb

## Action Authoring and Deployment
A Cortex Action can be authored and deployed from within a notebook for rapid prototyping and testing.  The Cortex SDK for Python includes an iPython Magic called `%%cortex_action` that takes a notebook cell and deploys it as a Cortex Action.

In [ ]:
%%cortex_action --name c12e/image-classify --function classify --requirements watson_developer_cloud

from cortex import Message
from cortex import Cortex
from watson_developer_cloud import VisualRecognitionV3

def classify(params):

    msg = Message(params)
    client = Cortex.client(api_endpoint=msg.apiEndpoint, token=msg.token)
    image_url = msg.payload.get('imageUrl')
    apiKey = msg.payload.get('apiKey')
    classifierId = msg.payload.get('classifierId')

    if classifierId is None or classifierId == "":
        return {"payload": "Classifier ID Not Provided"}

    visual_recognition = VisualRecognitionV3('2018-03-19', iam_apikey=apiKey)

    response = visual_recognition.classify(url=image_url, classifier_ids=[classifierId], threshold="0.5").get_result()

    if response is not None:
        return {"payload": response}
    else:
        return {"payload": "Unable to get response from Watson Visual Recognition"}

## Local Testing
Once the Action is built, we can run both a local and remote test.  The local test in our example can be done by simply calling the `detect` function we defined above.

In [ ]:
import sys
import argparse
from pprint import pprint


def test_detect(args):
    imageUrl = 'https://showmecables-static.scdn3.secure.raxcdn.com/media/catalog/product/cache/c687aa7517cf01e65c009f6943c2b1e9/u/s/usb-a-male-to-usb-b-female-adapter-3506-1.jpg'
    classifierId = 'Connectors_227384960'
    apiKey = 'BVaceevn9oDep6ow12MASoLubdwkLYkDYEKdjLdBKFRi'
    m = Message.with_payload({'imageUrl': imageUrl,'apiKey':apiKey,'classifierId':classifierId})
    result = classify(m.to_params())
    pprint(result['payload'])

test_detect(None)

## Remote Testing
Using the Cortex client, we can execute a remote test of our Action to make sure it deployed as expected.

In [ ]:
client = Cortex.client()
action = client.action('c12e/image-classify')
action

In [ ]:
imageUrl = 'https://showmecables-static.scdn3.secure.raxcdn.com/media/catalog/product/cache/c687aa7517cf01e65c009f6943c2b1e9/u/s/usb-a-male-to-usb-b-female-adapter-3506-1.jpg'
classifierId = 'Connectors_227384960'
apiKey = 'BVaceevn9oDep6ow12MASoLubdwkLYkDYEKdjLdBKFRi'
params = {'payload': {'imageUrl': imageUrl,'apiKey':apiKey,'classifierId':classifierId}}
rs = action.invoke(client.message(params.get('payload')))
result = rs.payload
classifiers =  result.get("images")
print (classifiers[0].get('classifiers'))